In [7]:
import torch
import numpy as np

## 1D Tensors

* supports indexex and slicing as usual python lists
* supports vector operations as np arrays, and operations with scalars and vectors (broadcasting)
* `torch.dot(,)` - dot product of two tensors

In [2]:
a = torch.tensor([1,2,3,4])
a.dtype, a.type()

(torch.int64, 'torch.LongTensor')

In [4]:
a.size(), a.ndimension()

(torch.Size([4]), 1)

In [6]:
a_col = a.view(-1, 1)
a_col.ndimension()

2

In [8]:
a = torch.from_numpy(np.array([1,2,3]))
a.type()

'torch.LongTensor'

In [9]:
a.tolist()

[1, 2, 3]

In [11]:
a = a.type(torch.FloatTensor)

In [12]:
a.mean(), a.std()

(tensor(2.), tensor(1.))

## 2D Tensors
* as in previous case, supports element-wise operations and operations with scalar
* `torch.mm(,)` - standart Matrix Multiplication

In [13]:
a = torch.tensor([[1,2,3], [4,5,6], [7,8,9]])
a.shape

torch.Size([3, 3])

In [14]:
a[0,0]

tensor(1)

## Deriviatives

$\frac{\partial}{\partial x} x^2 = 2 x$, therefore in case of $x=2$ deriviative equals to $4$.

In [22]:
x = torch.tensor(2., requires_grad=True)
y = x ** 2
y.backward()
x.grad

tensor(4.)

In [23]:
x = torch.tensor(2., requires_grad=True)
z = x ** 2 + 2 * x + 1
z.backward()
x.grad

tensor(6.)

In [25]:
class SQ(torch.autograd.Function):

    @staticmethod
    def forward(ctx,i):
        """
        In the forward pass we receive a Tensor containing the input and return
        a Tensor containing the output. ctx is a context object that can be used
        to stash information for backward computation. You can cache arbitrary
        objects for use in the backward pass using the ctx.save_for_backward method.
        """
        result=i**2
        ctx.save_for_backward(i)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        """
        In the backward pass we receive a Tensor containing the gradient of the loss
        with respect to the output, and we need to compute the gradient of the loss
        with respect to the input.
        """
        i, = ctx.saved_tensors
        grad_output = 2*i
        return grad_output

In [26]:
x=torch.tensor(2.0,requires_grad=True )
sq=SQ.apply

y=sq(x)
y
print(y.grad_fn)
y.backward()
x.grad

tensor(4.)

## Simple Dataset

* Useful in cases when we don't want to store whole dataset in memory, so we can load it in lazy fashion
* Can apply chain of transformations (pointwise)

In [29]:
from torch.utils.data import Dataset
from torchvision import transforms


class toy_set(Dataset):
    
    # Constructor with defult values 
    def __init__(self, length = 100, transform = None):
        self.len = length
        self.x = 2 * torch.ones(length, 2)
        self.y = torch.ones(length, 1)
        self.transform = transform
     
    # Getter
    def __getitem__(self, index):
        sample = self.x[index], self.y[index]
        if self.transform:
            sample = self.transform(sample)     
        return sample
    
    # Get Length
    def __len__(self):
        return self.len
    
    
class add_mult(object):
    
    # Constructor
    def __init__(self, addx = 1, muly = 2):
        self.addx = addx
        self.muly = muly
    
    # Executor
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x + self.addx
        y = y * self.muly
        sample = x, y
        return sample
    

class mult(object):
    
    # Constructor
    def __init__(self, mult = 100):
        self.mult = mult
        
    # Executor
    def __call__(self, sample):
        x = sample[0]
        y = sample[1]
        x = x * self.mult
        y = y * self.mult
        sample = x, y
        return sample

In [30]:
data_transform = transforms.Compose([add_mult(), mult()])

In [31]:
dataset = toy_set(10, transform=data_transform)
for line in dataset:
    print(line)

(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
(tensor([300., 300.]), tensor([200.]))
